# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
cust_prod = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()

In [5]:
cust_prod.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivCust_prod = cust_prod.pivot_table(values='Quantity', columns='CustomerID', 
                      index='ProductName', aggfunc='sum').fillna(0)

In [7]:
pivCust_prod.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
# Por producto

distances = pdist(pivCust_prod, 'euclidean')

matrix = squareform(distances)

In [9]:
matrix

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

In [10]:
# Por cliente

distances2 = pdist(pivCust_prod.T, 'euclidean')

matrix2 = squareform(distances2)

In [11]:
matrix2

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [12]:
relation = pd.DataFrame(1/(1 + matrix2),index=pivCust_prod.columns,columns=pivCust_prod.columns)

In [13]:
relation.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [14]:
# Similitudes con el cliente número 33

similarCust = relation[33].sort_values(ascending=False)[1:]

top5_cust33 = similarCust.head()

top5_cust33

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [15]:
# Hago una lista con los ID de los clientes similares al cliente 33

top5_cust33 = [customer for customer in top5_cust33.index]

In [16]:
similar_cust = cust_prod[cust_prod['CustomerID'].isin(top5_cust33)]

In [17]:
similar_cust.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
top_products = similar_cust.groupby('ProductName')['Quantity'].sum().reset_index()

In [19]:
top_products.head()

,ProductName,Quantity
0,Anchovy Paste - 56 G Tube,1
1,Appetizer - Mushroom Tart,1
2,Appetizer - Sausage Rolls,1
3,Apricots - Dried,1
4,Apricots - Halves,1


In [20]:
top_products = top_products.sort_values('Quantity',ascending=False).reset_index()

In [21]:
top_products.head()

,index,ProductName,Quantity
0,33,Butter - Unsalted,3
1,203,Wine - Ej Gallo Sierra Valley,3
2,186,Towels - Paper / Kraft,3
3,168,Soup - Campbells Bean Medley,3
4,199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [22]:
# Creo nueva tabla solo con los clientes parecidos y el cliente 33

prod_recommend = pivCust_prod[33]

In [23]:
# Fusiono la pivot table por producto con la tabla de productos por clientes parecidos

df_merged = pd.merge(prod_recommend, top_products, on='ProductName', how='left').fillna(0)

In [24]:
# Filtro la tabla por los productos que el cliente 33 no ha comprado

df_merged = df_merged.loc[df_merged[33] == 0]

In [25]:
# Compruebo que está bien filtrado

df_merged[33].value_counts()

0.0    393
Name: 33, dtype: int64

In [26]:
# Ordeno por los productos más comprados por los clientes parecidos

df_merged = df_merged.sort_values(by=['Quantity'],ascending=False)

In [27]:
# Top 5 productos recomendados

df_merged.head()

,ProductName,33,index,Quantity
421,Wine - Ej Gallo Sierra Valley,0.0,203.0,3.0
341,Soup - Campbells Bean Medley,0.0,168.0,3.0
73,Butter - Unsalted,0.0,33.0,3.0
415,Wine - Blue Nun Qualitatswein,0.0,199.0,3.0
193,Juice - Lime,0.0,89.0,2.0


In [28]:
df_merged.drop(['index'], axis=1).head()

,ProductName,33,Quantity
421,Wine - Ej Gallo Sierra Valley,0.0,3.0
341,Soup - Campbells Bean Medley,0.0,3.0
73,Butter - Unsalted,0.0,3.0
415,Wine - Blue Nun Qualitatswein,0.0,3.0
193,Juice - Lime,0.0,2.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [29]:
def recommendations():

    recommend = {}
    customers = list(data['CustomerID'].unique())

    for customer in customers:
        # lista de los top5 clientes relacionados
        similarCust = list(relation[customer].sort_values(ascending=False)[1:].head().index)
        # tabla solo con los top5 clientes y los productos comprados
        table_simCust = cust_prod[cust_prod['CustomerID'].isin(similarCust)]
        # agrupación por producto para ver cantidad total comprada por el top5 clientes
        top_products = table_simCust.groupby('ProductName')['Quantity'].sum().reset_index()
        # ordenado descendiente por cantidad total comprada de cada producto
        top_products = top_products.sort_values('Quantity',ascending=False).reset_index()
        # fusión tabla productos/clientes + top productos comprados por top5 clientes
        df_merged = pd.merge(pivCust_prod[customer], top_products, on='ProductName', how='left').fillna(0)
        # elimino la columna arratrada del index
        df_merged = df_merged.drop(['index'], axis=1)
        # filtro por los productos que el cliente a estudiar no ha comprado
        df_merged = df_merged.loc[df_merged[customer] == 0]
        # ordeno el top productos
        result = df_merged.sort_values('Quantity',ascending=False).head()
        # añado los resultados al dictionario
        recommend[customer] = list(result['ProductName'])
    return recommend

In [30]:
product_recommendations = pd.DataFrame(recommendations()).T
product_recommendations.head()

,0,1,2,3,4
61288,Chicken - Soup Base,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,Dc Hikiage Hira Huba
77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Beer - Sleemans Cream Ale,Chocolate - Feathers,Tuna - Salad Premix
23548,Squid - Tubes / Tenticles 10/20,Puree - Mocha,Pernod,Banana Turning,Flavouring - Orange
78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Beef - Inside Round,Pop Shoppe Cream Soda


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [31]:
product_recommendations = product_recommendations.reset_index()

In [32]:
product_recommendations.columns = ['Customer ID','Product1','Product2','Product3','Product4','Product5']

In [33]:
product_recommendations.head()

,Customer ID,Product1,Product2,Product3,Product4,Product5
0,61288,Chicken - Soup Base,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,Dc Hikiage Hira Huba
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Beer - Sleemans Cream Ale,Chocolate - Feathers,Tuna - Salad Premix
3,23548,Squid - Tubes / Tenticles 10/20,Puree - Mocha,Pernod,Banana Turning,Flavouring - Orange
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Beef - Inside Round,Pop Shoppe Cream Soda


In [34]:
# Creo función para automatizar estos últimos pasos

def dataframe():
    product_recommendations = pd.DataFrame(recommendations()).T
    product_recommendations = product_recommendations.reset_index()
    product_recommendations.columns = ['Customer ID','Product1','Product2','Product3','Product4','Product5']
    return product_recommendations

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [36]:
# Valores a cambiar

distances2 = pdist(pivCust_prod.T, 'correlation')
matrix2 = squareform(distances2)
relation = pd.DataFrame(1/(1 + matrix2),index=pivCust_prod.columns,columns=pivCust_prod.columns)

In [37]:
# llamo a la última función creada para que haga todo el proceso de una

final = dataframe()
final.head()

,Customer ID,Product1,Product2,Product3,Product4,Product5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Fondant - Icing,Grenadine,Mangoes,Veal - Osso Bucco,Lettuce - Frisee
2,40094,Truffle Cups - Brown,"Bread - Roll, Soft White Round",Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
3,23548,Olives - Stuffed,Wine - Charddonnay Errazuriz,Soup - Campbells Tomato Ravioli,Assorted Desserts,Juice - Orange
4,78981,"Tart Shells - Sweet, 4",Sprouts - Baby Pea Tendrils,Artichokes - Jerusalem,"Appetizer - Mini Egg Roll, Shrimp",Chocolate - Dark
